<a href="https://colab.research.google.com/github/marciamart/Classificacao-Ecommerce/blob/main/classifica%C3%A7%C3%A3o_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/fatihkgg/ecommerce-product-images-18k")

#USAR API TOKEN DO KAGGLE

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mrciamartins
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/fatihkgg/ecommerce-product-images-18k


100%|██████████| 142M/142M [00:02<00:00, 59.9MB/s]


Dataset em  /content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES

## Parte 1

### Objetivo e informações esperadas

Objetivo:
* Apresentar informações do dataset.

Informações esperadas:
* Integridade dos Arquivos
  1. Verifique se todas as imagens listadas no arquivo de informações
realmente existem no diretório de imagens e vice-versa.
  2. Verifique se todas as imagens estão no mesmo formato, ex: JPEG,
PNG, etc.
* Consistência dos Metadados
  1. Verifique se há valores ausentes nos metadados e como esses casos
são tratados.
  2. Verifique valores inconsistentes, por exemplo: dimensões de imagens
fora do esperado.
* Qualidade das Imagens
  1. Identifique imagens corrompidas que não podem ser abertas ou
processadas.
* Distribuição das Classes
  1. Verifique a distribuição das classes para identificar possíveis
desequilíbrios que possam afetar a modelagem
* Duplicatas
  1. Identifique imagens duplicadas que possam enviesar os resultados.
  2. Verifique duplicatas no arquivo de informações.

Informações complementares:
* Elaborar uma apresentação para mostrar os resultados.
* Apresentar o dataset de forma detalhada.
* Verifique quais itens em Informações esperadas podem ser aplicados no
dataset.

### **Integridade dos Arquivos**
A função a seguir verifica se todas as imagens no diretório estão no mesmo formato (JPEG, JPG, PNG). Ela retorna um dicionário com os formatos das imagens como chaves e suas respectivas quantidades como valores.

Saída esperada: {'JPEG': 18175}

In [ ]:
import os
from PIL import Image

diretorio_base = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES'

def verificar_formato_imagem(diretorio_base):
    formatos = {}
    for raiz, dirs, arquivos in os.walk(diretorio_base):
        for arquivo in arquivos:
            if arquivo.endswith(('jpeg', 'jpg', 'png')):
                caminho_arquivo = os.path.join(raiz, arquivo)
                with Image.open(caminho_arquivo) as img:
                    formato = img.format
                    if formato not in formatos:
                        formatos[formato] = 0
                    formatos[formato] += 1
            else:
                formatos['Desconhecido'] = formatos.get('Desconhecido', 0) + 1

    return formatos

formatos_imagens = verificar_formato_imagem(diretorio_base)
print(formatos_imagens)

{'JPEG': 18175}


### **Distribuição das Classes**

Conjuntos de treinamento (train) e validação (val) para treinamento de modelo e ajuste de hiperparâmetros. E uma pequena amostra, conjunto de verificação (check) para avaliação visual do modelo durante a implantação.

Aqui faremos uma análise da distribuição das imagens nessas classes.

#### **TRAIN**

In [ ]:
import os
import pandas as pd

#análise da distribuição da classe: TRAIN
data =  '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/train'
subdiretorios = {}
# Passando por cada subdiretório
for dirpath, dirnames, filenames in os.walk(data):
    if dirpath == data: continue

    num_images = 0
    # Conta o número de imagens no subdiretório
    for filename in filenames:
        if filename.lower().endswith('.jpeg'):
            num_images += 1

    # Imprimi o número de imagens do subdiretório atual
    print(f"São {num_images} imagens no diretório {os.path.basename(dirpath)}")
    subdiretorios[os.path.basename(dirpath)] = num_images
  
print(f"Subdiretórios: {subdiretorios}")

In [ ]:
import matplotlib.pyplot as plt

# Criando um gráfico de barras para visualização da distribuição das classes
plt.bar(subdiretorios.keys(), subdiretorios.values())
plt.xticks(rotation=90)
plt.ylabel("Quantidade de Imagens")
plt.title("Distribuição das Classes de Treinamento")
plt.show()